In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
import wiki
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

In [3]:
PROPHET_PATH = '../data/prophet/'
RESULTS_PATH = 'results/'

In [4]:
pagedf = pd.read_feather(PROPHET_PATH+'pagedf.f')
ds = pd.read_feather(PROPHET_PATH+'ds.f')

## Version 1.3f
Should set version directory name in next cell. Should describe version specifics (outliers, holidays, validation period)

* Val indexing on -60
* No outlier fixing
* Linear growth
* Truncating predictions at 0
* Fill ALL NaNs to 0

### Remarks
* ?

In [5]:
# should break if the dir already exists - avoids accidental overwriting
VERSION = 'v1.3f/'
val_lims = (0,-60)
os.makedirs(PROPHET_PATH+VERSION)

In [6]:
pagedf = pagedf.fillna(0); pagedf.head()

,0,1,2,3,4,5,6,7,8,9,...,145053,145054,145055,145056,145057,145058,145059,145060,145061,145062
0,18.0,11.0,1.0,35.0,0.0,12.0,0.0,118.0,5.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11.0,14.0,0.0,13.0,0.0,7.0,0.0,26.0,23.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,15.0,1.0,10.0,0.0,4.0,0.0,30.0,14.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13.0,18.0,1.0,94.0,0.0,5.0,0.0,24.0,12.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14.0,11.0,0.0,4.0,0.0,20.0,0.0,29.0,9.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# WARNING:
Turned off the chained assignment warning - when slicing dfs they can return copies sometimes instead,
which will mean your assignment wont be done on the actual base df.
Not sure why it's still compaining at me when I'm using .loc for assignations everywhere... shitty

In [7]:
pd.options.mode.chained_assignment = None

In [8]:
def process_page(page):
    df = ds.join(pagedf[page])
    df.columns = ['ds','y']
    # note this is doing validation on last 60 days
    # should also consider doing validation on the time period we are forcasting
    traindf = df.iloc[val_lims[0]:val_lims[1]]
    traindf['train'] = 1 # feather won't serialize bool so 1s and 0s...
    # do outlier removal here
    #traindf.loc[traindf.y > traindf.y.quantile(.95), ['y']] = None
    m = Prophet(yearly_seasonality=True)
    m.fit(traindf)
    forecast = m.predict(ds)
    forecast['yhat_org'] = forecast['yhat']
    forecast.loc[forecast['yhat'] < 0,['yhat']] = 0.0
    forecast = forecast.join(df.y)
    forecast = forecast.join(traindf.loc[:,['train']]).fillna({'train':0}) # 0 bools
    forecast.to_feather(PROPHET_PATH+VERSION+page+'df.f')
    with open(PROPHET_PATH+VERSION+page+'m.pk', 'wb') as file:
        pk.dump(m,file)
    full_smape = wiki.val.smape(forecast.y, forecast.yhat)
    val_smape = wiki.val.smape(forecast[forecast['train'] == 0].y,forecast[forecast['train'] == 0].yhat)
    return (page, full_smape, val_smape)

In [14]:
def wrapper(pages):
    val_results = []
    for page in pages:
        print(page,)
        val_results.append(process_page(page))
    return val_results

In [15]:
total_proc = 20

In [16]:
col_split = np.array_split(pagedf.columns, total_proc)
mp_pool = mp.Pool(total_proc)

0
7254
14508
21762
29015
36268
43521
50774
58027
65280
72533
79786
87039
94292
101545
108798
123304
130557
116051
137810
14509
101546
1
65281
123305
137811
7255
72534
29016
79787
108799
87040
50775
58028
43522
130558
94293
116052
36269
21763
58029
14510
137812
94294
65282
101547
36270
87041
108800
50776
123306
72535
29017
79788
116053
130559
2
7256
43523
21764


Process ForkPoolWorker-57:
Process ForkPoolWorker-54:
Process ForkPoolWorker-59:
Process ForkPoolWorker-48:
Process ForkPoolWorker-53:
Process ForkPoolWorker-46:
Traceback (most recent call last):
Process ForkPoolWorker-41:
Process ForkPoolWorker-42:
Process ForkPoolWorker-51:
Process ForkPoolWorker-44:
Process ForkPoolWorker-50:
Process ForkPoolWorker-60:
Process ForkPoolWorker-52:
Process ForkPoolWorker-43:
Process ForkPoolWorker-56:
Process ForkPoolWorker-47:
Process ForkPoolWorker-55:
Process ForkPoolWorker-58:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Tracebac

  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-14-1d87e76f5d54>", line 5, in wrapper
    val_results.append(process_page(page))
  File "<ipython-input-14-1d87e76f5d54>", line 5, in wrapper
    val_results.append(process_page(page))
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/multiprocessing/pool.py", line 44, 

  File "<ipython-input-14-1d87e76f5d54>", line 5, in wrapper
    val_results.append(process_page(page))
  File "<ipython-input-8-e06a84e7f481>", line 12, in process_page
    forecast = m.predict(ds)
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/fbprophet/forecaster.py", line 568, in predict
    intervals = self.predict_uncertainty(df)
KeyboardInterrupt
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/fbprophet/forecaster.py", line 567, in predict
    seasonal_components = self.predict_seasonal_components(df)
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/fbprophet/forecaster.py", line 729, in predict_uncertainty
    sim = self.sample_model(df, seasonal_features, i)
  File "<ipython-input-14-1d87e76f5d54>", line 5, in wrapper
    val_results.append(process_page(page))
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/nanfunctions.py", line 1143, in _nanpercentile
    overwrite_input

  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/nanfunctions.py", line 1120, in nanpercentile
    interpolation=interpolation)
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/fbprophet/forecaster.py", line 724, in predict_uncertainty
    seasonal_features = self.make_all_seasonality_features(df)
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/pandas/core/base.py", line 197, in _getter
    return self._delegate_property_get(name)
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/fbprophet/forecaster.py", line 823, in sample_predictive_trend
    trend = self.piecewise_linear(t, deltas, k, m, changepoint_ts)
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py", line 3858, in _ureduce
    r = func(a, **kwargs)
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/fbprophet/forecaster.py", line 729, in predict_uncertainty
  

  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py", line 4116, in percentile
    interpolation=interpolation)
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/shape_base.py", line 107, in apply_along_axis
    res = func1d(arr[tuple(i.tolist())], *args, **kwargs)
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/pandas/core/indexes/timedeltas.py", line 394, in days
    return self._get_field('days')
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py", line 4164, in _percentile
    indices = q * (Nx - 1)
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py", line 4233, in _percentile
    x1 = take(ap, indices_below, axis=axis) * weights_below
  File "/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/nanfunctions.py", line 1172, in _nanpercentile1d
    interpolation=inte

58030
58031
58032
58033
58034
58035
58036
58037
58038
58039
58040
58041
58042
58043
58044
58045
58046
58047
58048
58049
58050
58051
58052
58053
58054
58055
58056
58057
58058
58059
58060
58061
58062
58063
58064
58065
58066
58067
58068
58069
58070
58071
58072
58073
58074
58075
58076
58077
58078
58079
58080
58081
58082
58083
58084
58085
58086
58087
58088
58089
58090
58091
58092
58093
58094
58095
58096
58097
58098
58099
58100
58101
58102
58103
58104
58105
58106
58107
58108
58109
58110
58111
58112
58113
58114
58115
58116
58117
58118
58119
58120
58121
58122
58123
58124
58125
58126
58127
58128
58129
58130
58131
58132
58133
58134
58135
58136
58137
58138
58139
58140
58141
58142
58143
58144
58145
58146
58147
58148
58149
58150
58151
58152
58153
58154
58155
58156
58157
58158
58159
58160
58161
58162
58163
58164
58165
58166
58167
58168
58169
58170
58171
58172
58173
58174
58175
58176
58177
58178
58179
58180
58181
58182
58183
58184
58185
58186
58187
58188
58189
58190
58191
58192
58193
58194
58195
5819

In [17]:
with utils.clock():
    %time val_results = mp_pool.map(wrapper, col_split)

KeyboardInterrupt: 

Elapsed time 19.5601487159729 seconds


In [ ]:
val_results = [item for sublist in val_results for item in sublist]
val_results = pd.DataFrame(val_results, columns=['page_index',VERSION[:-1]+'_full',VERSION[:-1]+'_val'])

In [ ]:
val_results.to_feather(PROPHET_PATH+RESULTS_PATH+VERSION[:-1]+'df.f')